In [1]:
mystr = '[UnityCrossThreadLogger]STATE CHANGED {"old":"ConnectedToMatchDoor_ConnectedToGRE_Waiting","new":"Playing"}'
mystr.split('[UnityCrossThreadLogger]')

['',
 'STATE CHANGED {"old":"ConnectedToMatchDoor_ConnectedToGRE_Waiting","new":"Playing"}']

In [2]:
mystr2 = 'STATE CHANGED {"old":"ConnectedToMatchDoor_ConnectedToGRE_Waiting","new":"Playing"}'
mystr2.split('[UnityCrossThreadLogger]')

['STATE CHANGED {"old":"ConnectedToMatchDoor_ConnectedToGRE_Waiting","new":"Playing"}']

In [3]:
len('[UnityCrossThreadLogger]')

24

In [1]:
import pandas as pd
import json

In [2]:
path = '/home/r3gal/develop/mtga_pipeline/data/parsed_csv'
dtype_map = {
    'game_num': 'Int64',
    'timestamp': 'str',
    'event': 'str',
    'payload': 'str'
}
df = pd.read_csv(f"{path}/Filtered_Player_20260212_233143_test.csv", dtype=dtype_map)
df['payload'] = df['payload'].apply(json.loads)
# df['payload'] = df['payload'].apply(pd.json_normalize)
df.head()

,game_num,player_id,timestamp,event,payload
0,1,V7JT5YS7ANCWRHG35IZNV4PKOY,2/12/2026 11:26:17 PM,GreToClientEvent,"[{'type': 'GREMessageType_DieRollResultsResp',..."
1,1,V7JT5YS7ANCWRHG35IZNV4PKOY,2/12/2026 11:26:19 PM,GreToClientEvent,"[{'type': 'GREMessageType_SetSettingsResp', 's..."
2,1,V7JT5YS7ANCWRHG35IZNV4PKOY,2/12/2026 11:26:20 PM,GreToClientEvent,"[{'type': 'GREMessageType_GameStateMessage', '..."
3,1,V7JT5YS7ANCWRHG35IZNV4PKOY,2/12/2026 11:26:29 PM,GreToClientEvent,"[{'type': 'GREMessageType_GameStateMessage', '..."
4,1,V7JT5YS7ANCWRHG35IZNV4PKOY,2/12/2026 11:26:40 PM,GreToClientEvent,"[{'type': 'GREMessageType_UIMessage', 'systemS..."


In [3]:
row = df.iloc[0]
payload = row['payload']
payload

[{'type': 'GREMessageType_DieRollResultsResp',
  'systemSeatIds': [1, 2],
  'msgId': 3,
  'dieRollResultsResp': {'playerDieRolls': [{'systemSeatId': 1,
     'rollValue': 7},
    {'systemSeatId': 2, 'rollValue': 2}]}},
 {'type': 'GREMessageType_GameStateMessage',
  'systemSeatIds': [2],
  'msgId': 4,
  'gameStateId': 1,
  'gameStateMessage': {'type': 'GameStateType_Full',
   'gameStateId': 1,
   'gameInfo': {'matchID': '2232b124-0796-47ba-bfb5-727f0d507814',
    'gameNumber': 1,
    'stage': 'GameStage_Start',
    'type': 'GameType_Duel',
    'variant': 'GameVariant_Normal',
    'matchState': 'MatchState_GameInProgress',
    'matchWinCondition': 'MatchWinCondition_SingleElimination',
    'maxTimeoutCount': 4,
    'maxPipCount': 3,
    'timeoutDurationSec': 30,
    'superFormat': 'SuperFormat_Constructed',
    'mulliganType': 'MulliganType_London',
    'deckConstraintInfo': {'minDeckSize': 60,
     'maxDeckSize': 250,
     'maxSideboardSize': 15}},
   'teams': [{'id': 1, 'playerIds': [1]

In [61]:
def safe_json_load(s):
    try:
        return json.loads(s)
    except (json.JSONDecodeError, TypeError):
        # Return empty dict if invalid
        return {}
payload_parsed = df['payload'].apply(safe_json_load)

# Flatten nested JSON into columns
payload_expanded = pd.json_normalize(payload_parsed)
# payload_expanded.head()
df.columns

Index(['game_num', 'metadata', 'payload'], dtype='str')

In [62]:

df_2 = df[~df['metadata'].str.contains('GreToClientEvent', na=False)]
# df_2.iloc[13]
df_2

,game_num,metadata,payload
170,1,2/12/2026 11:31:11 PM: Match to V7JT5YS7ANCWRHG35IZNV4PKOY: MatchGameRoomStateChangedEvent\n,"{ ""transactionId"": ""f192ba64-9c7b-4e34-96a2-1cf477e5e9e0"", ""requestId"": 103, ""timestamp"": ""1770957070891"", ""matchGameRoomStateChangedEvent"": { ""gameRoomInfo"": { ""gameRoomConfig"": { ""reservedPlayers"": [ { ""userId"": ""INPWZ5Y6HBC7ZBIBMGAIE66D6M"", ""playerName"":""Bob"", ""systemSeatId"": 1, ""teamId"": 1, ""courseId"": ""Avatar_Basic_Rydia_FIN"", ""sessionId"": ""abf944a0-a239-49c5-9458-576bd6c6b978"", ""platformId"": ""iPhone"", ""eventId"": ""Play"" }, { ""userId"": ""V7JT5YS7ANCWRHG35IZNV4PKOY"", ""playerName"":""R3GAL"", ""systemSeatId"": 2, ""teamId"": 2, ""courseId"": ""Avatar_Basic_JaceBeleren"", ""sessionId"": ""e2d82f81-8d85-434b-8830-fd57a4201b9e"", ""platformId"": ""Windows"", ""eventId"": ""Play"" } ], ""matchId"": ""2232b124-0796-47ba-bfb5-727f0d507814"" }, ""stateType"": ""MatchGameRoomStateType_MatchCompleted"", ""finalMatchResult"": { ""matchId"": ""2232b124-0796-47ba-bfb5-727f0d507814"", ""matchCompletedReason"": ""MatchCompletedReasonType_Success"", ""resultList"": [ { ""scope"": ""MatchScope_Game"", ""result"": ""ResultType_WinLoss"", ""winningTeamId"": 1, ""reason"": ""ResultReason_Game"" }, { ""scope"": ""MatchScope_Match"", ""result"": ""ResultType_WinLoss"", ""winningTeamId"": 1, ""reason"": ""ResultReason_Game"" } ] } } } }\n"
171,1,STATE CHANGED,"{""old"":""Playing"",""new"":""MatchCompleted""}\n"
172,1,GREConnection.HandleWebSocketClosed,"{""closeType"":7,""reason"":""Closed by remote end"",""tcpConn"":{""host"":""matchdoor-mtga-production-2026-56-6-2.w2.mtgarena.com"",""port"":30003,""rtTicksRollingAvg"":73.0,""rtTicksSamples"":[82.0,82.0,67.0,67.0,67.0],""lastLocalActivity"":[{""type"":""Send"",""time"":""2026-02-13T04:31:02.0426496Z"",""msgType"":""ClientToMatchSvcMsgWrapper"",""msgVersion"":4},{""type"":""Send"",""time"":""2026-02-13T04:31:02.4876418Z"",""msgType"":""ClientToMatchSvcMsgWrapper"",""msgVersion"":4},{""type"":""Send"",""time"":""2026-02-13T04:31:03.1916294Z"",""msgType"":""ClientToMatchSvcMsgWrapper"",""msgVersion"":4},{""type"":""Send"",""time"":""2026-02-13T04:31:03.3626264Z"",""msgType"":""ClientToMatchSvcMsgWrapper"",""msgVersion"":4},{""type"":""Send"",""time"":""2026-02-13T04:31:04.1146132Z"",""msgType"":""ClientToMatchSvcMsgWrapper"",""msgVersion"":4},{""type"":""Send"",""time"":""2026-02-13T04:31:09.9375108Z"",""msgType"":""ClientToMatchSvcMsgWrapper"",""msgVersion"":4},{""type"":""Send"",""time"":""2026-02-13T04:31:09.9545105Z"",""msgType"":""ClientToMatchSvcMsgWrapper"",""msgVersion"":4},{""type"":""Send"",""time"":""2026-02-13T04:31:11.3674856Z"",""msgType"":""Pong"",""msgVersion"":4},{""type"":""Send"",""time"":""2026-02-13T04:31:11.4354844Z"",""msgType"":""Ping"",""msgVersion"":4},{""type"":""Send"",""time"":""2026-02-13T04:31:13.8014428Z"",""msgType"":""ClientToMatchSvcMsgWrapper"",""msgVersion"":4},{""type"":""Send"",""time"":""2026-02-13T04:31:13.8354422Z"",""msgType"":""ClientToMatchSvcMsgWrapper"",""msgVersion"":4},{""type"":""Close"",""time"":""2026-02-13T04:31:13.8704416Z"",""status"":7,""reason"":""Closed by remote end""}],""lastRemoteActivity"":[{""type"":""Msg"",""time"":""2026-02-13T04:30:59.7356902Z"",""msgVersion"":4},{""type"":""Msg"",""time"":""2026-02-13T04:31:01.055667Z"",""msgVersion"":4},{""type"":""Msg"",""time"":""2026-02-13T04:31:03.0556318Z"",""msgVersion"":4},{""type"":""Msg"",""time"":""2026-02-13T04:31:03.2616282Z"",""msgVersion"":4},{""type"":""Msg"",""time"":""2026-02-13T04:31:04.2356111Z"",""msgVersion"":4},{""type"":""Msg"",""time"":""2026-02-13T04:31:06.2345759Z"",""msgVersion"":4},{""type"":""Msg"",""time"":""2026-02-13T04:31:10.717497Z"",""msgVersion"":4},{""type"":""Msg"",""time"":""2026-02-13T04:31:10.9024938Z"",""msgVersion"":4},{""type"":""Msg"",""time"":""2026-02-13T04:31:10.9414931Z"",""msgVersion"":4},{""type"":""Ping"",""time"":""2026-02-13T04:31:11.3584858Z"",""msgVersion"":4},{""type""

In [14]:
len('[UnityCrossThreadLogger]==> DeckUpsertDeckV2 ')

45

In [15]:
line = '[UnityCrossThreadLogger]==> DeckUpsertDeckV2 {"id":"c559b3a9-2778-4b41-9c56-23fceb37520a","request":"{\"Summa'
index = line.find('{')
index

45

In [16]:
import duckdb

# Connect to persistent DuckDB database
con = duckdb.connect(database='mtga_local.duckdb')

In [1]:
import json

my_str = """
{ "transactionId": "5b74f40a-14e4-4f13-bd5f-c146debdd3e7", "requestId": 783, "timestamp": "1771110144901", "greToClientEvent": { "greToClientMessages": [ { "type": "GREMessageType_GameStateMessage", "systemSeatIds": [ 2 ], "msgId": 1067, "gameStateId": 744, "gameStateMessage": { "type": "GameStateType_Diff", "gameStateId": 744, "turnInfo": { "phase": "Phase_Main2", "turnNumber": 32, "activePlayer": 2, "priorityPlayer": 1, "decisionPlayer": 2, "nextPhase": "Phase_Ending", "nextStep": "Step_End" }, "prevGameStateId": 743, "timers": [ { "timerId": 9, "type": "TimerType_ActivePlayer", "durationSec": 121, "elapsedSec": 3, "behavior": "TimerBehavior_TakeControl", "warningThresholdSec": 30, "elapsedMs": 3759 }, { "timerId": 12, "type": "TimerType_Inactivity", "durationSec": 150, "behavior": "TimerBehavior_Timeout", "warningThresholdSec": 30 } ], "update": "GameStateUpdate_SendHiFi", "actions": [ { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 368, "abilityGrpId": 1004 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 340, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 358, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 375, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 437, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 444, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 505, "abilityGrpId": 1005 } }, { "seatId": 2, "action": { "actionType": "ActionType_Cast", "instanceId": 462, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 1 }, { "color": [ "ManaColor_Black" ], "count": 1 } ] } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 344, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 371, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 378, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 386, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 392, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 430, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 440, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 507, "abilityGrpId": 1152 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate", "instanceId": 507, "abilityGrpId": 175909, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 4, "abilityGrpId": 175909 } ] } } ] } }, { "type": "GREMessageType_GameStateMessage", "systemSeatIds": [ 2 ], "msgId": 1068, "gameStateId": 745, "gameStateMessage": { "type": "GameStateType_Diff", "gameStateId": 745, "turnInfo": { "phase": "Phase_Ending", "step": "Step_End", "turnNumber": 32, "activePlayer": 2, "priorityPlayer": 2, "decisionPlayer": 2, "nextPhase": "Phase_Ending", "nextStep": "Step_Cleanup" }, "annotations": [ { "id": 1648, "affectedIds": [ 2 ], "type": [ "AnnotationType_PhaseOrStepModified" ], "details": [ { "key": "phase", "type": "KeyValuePairValueType_int32", "valueInt32": [ 5 ] }, { "key": "step", "type": "KeyValuePairValueType_int32", "valueInt32": [ 0 ] } ] } ], "prevGameStateId": 744, "update": "GameStateUpdate_SendHiFi", "actions": [ { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 368, "abilityGrpId": 1004 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 340, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 358, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 375, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 437, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 444, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 505, "abilityGrpId": 1005 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 344, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 371, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 378, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 386, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 392, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 430, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 440, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 507, "abilityGrpId": 1152 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate", "instanceId": 507, "abilityGrpId": 175909, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 4, "abilityGrpId": 175909 } ] } }, { "seatId": 2, "action": { "actionType": "ActionType_Cast", "instanceId": 462, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 1 }, { "color": [ "ManaColor_Black" ], "count": 1 } ] } } ] } }, { "type": "GREMessageType_GameStateMessage", "systemSeatIds": [ 2 ], "msgId": 1069, "gameStateId": 746, "gameStateMessage": { "type": "GameStateType_Diff", "gameStateId": 746, "turnInfo": { "phase": "Phase_Ending", "step": "Step_End", "turnNumber": 32, "activePlayer": 2, "priorityPlayer": 1, "decisionPlayer": 2, "nextPhase": "Phase_Ending", "nextStep": "Step_Cleanup" }, "prevGameStateId": 745, "update": "GameStateUpdate_SendHiFi", "actions": [ { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 368, "abilityGrpId": 1004 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 340, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 358, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 375, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 437, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 444, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 505, "abilityGrpId": 1005 } }, { "seatId": 2, "action": { "actionType": "ActionType_Cast", "instanceId": 462, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 1 }, { "color": [ "ManaColor_Black" ], "count": 1 } ] } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 344, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 371, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 378, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 386, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 392, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 430, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 440, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 507, "abilityGrpId": 1152 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate", "instanceId": 507, "abilityGrpId": 175909, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 4, "abilityGrpId": 175909 } ] } } ] } }, { "type": "GREMessageType_GameStateMessage", "systemSeatIds": [ 2 ], "msgId": 1070, "gameStateId": 747, "gameStateMessage": { "type": "GameStateType_Diff", "gameStateId": 747, "players": [ { "lifeTotal": 3, "systemSeatNumber": 1, "status": "PlayerStatus_InGame", "maxHandSize": 7, "turnNumber": 17, "teamId": 1, "timerIds": [ 1, 2, 3, 4, 5, 6 ], "controllerSeatId": 1, "controllerType": "ControllerType_Player", "timeoutCount": 4, "startingLifeTotal": 20 } ], "turnInfo": { "phase": "Phase_Beginning", "step": "Step_Upkeep", "turnNumber": 33, "activePlayer": 1, "priorityPlayer": 1, "nextPhase": "Phase_Beginning", "nextStep": "Step_Draw" }, "zones": [ { "zoneId": 30, "type": "ZoneType_Limbo", "visibility": "Visibility_Public", "objectInstanceIds": [ 514, 530, 301, 525, 180 ] } ], "gameObjects": [ { "instanceId": 340, "grpId": 96191, "type": "GameObjectType_Card", "zoneId": 28, "visibility": "Visibility_Public", "ownerSeatId": 1, "controllerSeatId": 1, "superTypes": [ "SuperType_Basic" ], "cardTypes": [ "CardType_Land" ], "subtypes": [ "SubType_Forest" ], "name": 647, "overlayGrpId": 96191, "uniqueAbilities": [ { "id": 200, "grpId": 1005 } ] }, { "instanceId": 358, "grpId": 96191, "type": "GameObjectType_Card", "zoneId": 28, "visibility": "Visibility_Public", "ownerSeatId": 1, "controllerSeatId": 1, "superTypes": [ "SuperType_Basic" ], "cardTypes": [ "CardType_Land" ], "subtypes": [ "SubType_Forest" ], "name": 647, "overlayGrpId": 96191, "uniqueAbilities": [ { "id": 214, "grpId": 1005 } ] }, { "instanceId": 375, "grpId": 96191, "type": "GameObjectType_Card", "zoneId": 28, "visibility": "Visibility_Public", "ownerSeatId": 1, "controllerSeatId": 1, "superTypes": [ "SuperType_Basic" ], "cardTypes": [ "CardType_Land" ], "subtypes": [ "SubType_Forest" ], "name": 647, "overlayGrpId": 96191, "uniqueAbilities": [ { "id": 230, "grpId": 1005 } ] }, { "instanceId": 526, "grpId": 93820, "type": "GameObjectType_Card", "zoneId": 28, "visibility": "Visibility_Public", "ownerSeatId": 1, "controllerSeatId": 1, "cardTypes": [ "CardType_Creature" ], "subtypes": [ "SubType_Elemental", "SubType_Hydra" ], "color": [ "CardColor_Green" ], "power": { "value": 1 }, "toughness": { "value": 1 }, "name": 823475, "overlayGrpId": 93820, "skinCode": "FF", "baseSkinCode": "FF", "uniqueAbilities": [ { "id": 367, "grpId": 14 }, { "id": 368, "grpId": 90859 }, { "id": 369, "grpId": 175873 } ] } ], "annotations": [ { "id": 1649, "affectedIds": [ 2 ], "type": [ "AnnotationType_PhaseOrStepModified" ], "details": [ { "key": "phase", "type": "KeyValuePairValueType_int32", "valueInt32": [ 5 ] }, { "key": "step", "type": "KeyValuePairValueType_int32", "valueInt32": [ 10 ] } ] }, { "id": 1650, "affectedIds": [ 0 ], "type": [ "AnnotationType_PhaseOrStepModified" ], "details": [ { "key": "phase", "type": "KeyValuePairValueType_int32", "valueInt32": [ 1 ] }, { "key": "step", "type": "KeyValuePairValueType_int32", "valueInt32": [ 0 ] } ] }, { "id": 1651, "affectorId": 1, "affectedIds": [ 1 ], "type": [ "AnnotationType_NewTurnStarted" ] }, { "id": 1652, "affectedIds": [ 1 ], "type": [ "AnnotationType_PhaseOrStepModified" ], "details": [ { "key": "phase", "type": "KeyValuePairValueType_int32", "valueInt32": [ 1 ] }, { "key": "step", "type": "KeyValuePairValueType_int32", "valueInt32": [ 1 ] } ] }, { "id": 1653, "affectedIds": [ 340 ], "type": [ "AnnotationType_TappedUntappedPermanent" ], "details": [ { "key": "tapped", "type": "KeyValuePairValueType_int32", "valueInt32": [ 0 ] } ] }, { "id": 1654, "affectedIds": [ 358 ], "type": [ "AnnotationType_TappedUntappedPermanent" ], "details": [ { "key": "tapped", "type": "KeyValuePairValueType_int32", "valueInt32": [ 0 ] } ] }, { "id": 1655, "affectedIds": [ 375 ], "type": [ "AnnotationType_TappedUntappedPermanent" ], "details": [ { "key": "tapped", "type": "KeyValuePairValueType_int32", "valueInt32": [ 0 ] } ] }, { "id": 1656, "affectedIds": [ 1 ], "type": [ "AnnotationType_PhaseOrStepModified" ], "details": [ { "key": "phase", "type": "KeyValuePairValueType_int32", "valueInt32": [ 1 ] }, { "key": "step", "type": "KeyValuePairValueType_int32", "valueInt32": [ 2 ] } ] } ], "diffDeletedInstanceIds": [ 300, 492, 516, 520 ], "prevGameStateId": 746, "timers": [ { "timerId": 4, "type": "TimerType_NonActivePlayer", "durationSec": 45, "behavior": "TimerBehavior_TakeControl", "warningThresholdSec": 30 }, { "timerId": 9, "type": "TimerType_ActivePlayer", "durationSec": 61, "behavior": "TimerBehavior_TakeControl", "warningThresholdSec": 30 } ], "update": "GameStateUpdate_SendHiFi", "actions": [ { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 368, "abilityGrpId": 1004 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 340, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 358, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 375, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 437, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 444, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 505, "abilityGrpId": 1005 } }, { "seatId": 2, "action": { "actionType": "ActionType_Cast", "instanceId": 462, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 1 }, { "color": [ "ManaColor_Black" ], "count": 1 } ] } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 344, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 371, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 378, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 386, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 392, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 430, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 440, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 507, "abilityGrpId": 1152 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate", "instanceId": 507, "abilityGrpId": 175909, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 4, "abilityGrpId": 175909 } ] } } ], "diffDeletedPersistentAnnotationIds": [ 5, 11 ] } }, { "type": "GREMessageType_GameStateMessage", "systemSeatIds": [ 2 ], "msgId": 1071, "gameStateId": 748, "gameStateMessage": { "type": "GameStateType_Diff", "gameStateId": 748, "turnInfo": { "phase": "Phase_Beginning", "step": "Step_Upkeep", "turnNumber": 33, "activePlayer": 1, "priorityPlayer": 2, "nextPhase": "Phase_Beginning", "nextStep": "Step_Draw" }, "prevGameStateId": 747, "update": "GameStateUpdate_SendHiFi", "actions": [ { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 368, "abilityGrpId": 1004 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 340, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 358, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 375, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 437, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 444, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 505, "abilityGrpId": 1005 } }, { "seatId": 2, "action": { "actionType": "ActionType_Cast", "instanceId": 462, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 1 }, { "color": [ "ManaColor_Black" ], "count": 1 } ] } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 344, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 371, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 378, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 386, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 392, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 430, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 440, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 507, "abilityGrpId": 1152 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate", "instanceId": 507, "abilityGrpId": 175909, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 4, "abilityGrpId": 175909 } ] } } ] } }, { "type": "GREMessageType_GameStateMessage", "systemSeatIds": [ 2 ], "msgId": 1072, "gameStateId": 749, "gameStateMessage": { "type": "GameStateType_Diff", "gameStateId": 749, "turnInfo": { "phase": "Phase_Beginning", "step": "Step_Draw", "turnNumber": 33, "activePlayer": 1, "priorityPlayer": 1, "nextPhase": "Phase_Main1" }, "zones": [ { "zoneId": 30, "type": "ZoneType_Limbo", "visibility": "Visibility_Public", "objectInstanceIds": [ 181, 514, 530, 301, 525, 180 ] }, { "zoneId": 31, "type": "ZoneType_Hand", "visibility": "Visibility_Private", "ownerSeatId": 1, "objectInstanceIds": [ 534 ], "viewers": [ 1 ] }, { "zoneId": 32, "type": "ZoneType_Library", "visibility": "Visibility_Hidden", "ownerSeatId": 1, "objectInstanceIds": [ 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218 ] } ], "annotations": [ { "id": 1657, "affectedIds": [ 1 ], "type": [ "AnnotationType_PhaseOrStepModified" ], "details": [ { "key": "phase", "type": "KeyValuePairValueType_int32", "valueInt32": [ 1 ] }, { "key": "step", "type": "KeyValuePairValueType_int32", "valueInt32": [ 3 ] } ] }, { "id": 1658, "affectedIds": [ 181 ], "type": [ "AnnotationType_ObjectIdChanged" ], "details": [ { "key": "orig_id", "type": "KeyValuePairValueType_int32", "valueInt32": [ 181 ] }, { "key": "new_id", "type": "KeyValuePairValueType_int32", "valueInt32": [ 534 ] } ] }, { "id": 1659, "affectedIds": [ 534 ], "type": [ "AnnotationType_ZoneTransfer" ], "details": [ { "key": "zone_src", "type": "KeyValuePairValueType_int32", "valueInt32": [ 32 ] }, { "key": "zone_dest", "type": "KeyValuePairValueType_int32", "valueInt32": [ 31 ] }, { "key": "category", "type": "KeyValuePairValueType_string", "valueString": [ "Draw" ] } ] } ], "prevGameStateId": 748, "update": "GameStateUpdate_SendHiFi", "actions": [ { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 368, "abilityGrpId": 1004 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 340, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 358, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 375, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 437, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 444, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 505, "abilityGrpId": 1005 } }, { "seatId": 2, "action": { "actionType": "ActionType_Cast", "instanceId": 462, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 1 }, { "color": [ "ManaColor_Black" ], "count": 1 } ] } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 344, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 371, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 378, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 386, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 392, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 430, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 440, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 507, "abilityGrpId": 1152 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate", "instanceId": 507, "abilityGrpId": 175909, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 4, "abilityGrpId": 175909 } ] } } ], "persistentAnnotations": [ { "id": 9, "affectorId": 31, "affectedIds": [ 534 ], "type": [ "AnnotationType_EnteredZoneThisTurn" ] } ] } }, { "type": "GREMessageType_GameStateMessage", "systemSeatIds": [ 2 ], "msgId": 1073, "gameStateId": 750, "gameStateMessage": { "type": "GameStateType_Diff", "gameStateId": 750, "turnInfo": { "phase": "Phase_Beginning", "step": "Step_Draw", "turnNumber": 33, "activePlayer": 1, "priorityPlayer": 2, "nextPhase": "Phase_Main1" }, "prevGameStateId": 749, "update": "GameStateUpdate_SendHiFi", "actions": [ { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 368, "abilityGrpId": 1004 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 340, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 358, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 375, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 437, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 444, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 505, "abilityGrpId": 1005 } }, { "seatId": 2, "action": { "actionType": "ActionType_Cast", "instanceId": 462, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 1 }, { "color": [ "ManaColor_Black" ], "count": 1 } ] } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 344, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 371, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 378, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 386, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 392, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 430, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 440, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 507, "abilityGrpId": 1152 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate", "instanceId": 507, "abilityGrpId": 175909, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 4, "abilityGrpId": 175909 } ] } } ] } }, { "type": "GREMessageType_GameStateMessage", "systemSeatIds": [ 2 ], "msgId": 1074, "gameStateId": 751, "gameStateMessage": { "type": "GameStateType_Diff", "gameStateId": 751, "turnInfo": { "phase": "Phase_Main1", "turnNumber": 33, "activePlayer": 1, "priorityPlayer": 1, "decisionPlayer": 1, "nextPhase": "Phase_Combat", "nextStep": "Step_BeginCombat" }, "annotations": [ { "id": 1660, "affectedIds": [ 1 ], "type": [ "AnnotationType_PhaseOrStepModified" ], "details": [ { "key": "phase", "type": "KeyValuePairValueType_int32", "valueInt32": [ 2 ] }, { "key": "step", "type": "KeyValuePairValueType_int32", "valueInt32": [ 0 ] } ] } ], "prevGameStateId": 750, "timers": [ { "timerId": 5, "type": "TimerType_Delay", "durationSec": 2, "running": true, "behavior": "TimerBehavior_StartDelayedTimer", "elapsedMs": 1 } ], "update": "GameStateUpdate_SendAndRecord", "actions": [ { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 368, "abilityGrpId": 1004 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 340, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 358, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 375, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 437, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 444, "abilityGrpId": 1005 } }, { "seatId": 1, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 505, "abilityGrpId": 1005 } }, { "seatId": 2, "action": { "actionType": "ActionType_Cast", "instanceId": 462, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 1 }, { "color": [ "ManaColor_Black" ], "count": 1 } ] } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 344, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 371, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 378, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 386, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 392, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 430, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 440, "abilityGrpId": 1003 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate_Mana", "instanceId": 507, "abilityGrpId": 1152 } }, { "seatId": 2, "action": { "actionType": "ActionType_Activate", "instanceId": 507, "abilityGrpId": 175909, "manaCost": [ { "color": [ "ManaColor_Generic" ], "count": 4, "abilityGrpId": 175909 } ] } } ] } } ] } }
"""
json.loads(my_str)


{'transactionId': '5b74f40a-14e4-4f13-bd5f-c146debdd3e7',
 'requestId': 783,
 'timestamp': '1771110144901',
 'greToClientEvent': {'greToClientMessages': [{'type': 'GREMessageType_GameStateMessage',
    'systemSeatIds': [2],
    'msgId': 1067,
    'gameStateId': 744,
    'gameStateMessage': {'type': 'GameStateType_Diff',
     'gameStateId': 744,
     'turnInfo': {'phase': 'Phase_Main2',
      'turnNumber': 32,
      'activePlayer': 2,
      'priorityPlayer': 1,
      'decisionPlayer': 2,
      'nextPhase': 'Phase_Ending',
      'nextStep': 'Step_End'},
     'prevGameStateId': 743,
     'timers': [{'timerId': 9,
       'type': 'TimerType_ActivePlayer',
       'durationSec': 121,
       'elapsedSec': 3,
       'behavior': 'TimerBehavior_TakeControl',
       'warningThresholdSec': 30,
       'elapsedMs': 3759},
      {'timerId': 12,
       'type': 'TimerType_Inactivity',
       'durationSec': 150,
       'behavior': 'TimerBehavior_Timeout',
       'warningThresholdSec': 30}],
     'update'